In [1]:
import json
import pandas as pd
import numpy as np
import ast
# import spacy
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

In [2]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict

In [3]:
from utils.track_1_kp_matching import *

/home/an/anaconda3/envs/deep_learning_env_2/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
import torch

In [5]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader

First with top 10 args per category labelling of test set

# Load Test dataset

In [6]:
df = pd.read_pickle(f"../data/Evaluation/test_data.pkl")

In [7]:
df

,HITId,comment_id,label,full_key_point,key_point_id,comment,full_comment,attributes,topic,isMultiAspect,...,old_key_point_id,key_point,aspects_x,opinions_x,opinion_aspect_pairs_x,sentiments_x,aspects_y,opinions_y,opinion_aspect_pairs_y,sentiments_y
0,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,1,Terrible administration and management .,kp_0_2,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,...,kp_0_2,terrible administration <SEP> negative,"[experience, employees]","[uninviting, rude]","[uninviting experience, rude employees]","[negative, negative]",administration,terrible,['terrible administration'],['negative']
1,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,1,Rude and Unprofessional staff .,kp_0_24,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,...,kp_0_24,unprofessional staff <SEP> negative,"[experience, employees]","[uninviting, rude]","[uninviting experience, rude employees]","[negative, negative]",staff,unprofessional,['unprofessional staff'],['negative']
2,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,0,Great for conventions and concerts ! !,kp_0_4,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,...,kp_0_4,great for conventions <SEP> positive,"[experience, employees]","[uninviting, rude]","[uninviting experience, rude employees]","[negative, negative]",conventions,great for,['great for conventions'],['positive']
3,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,0,Horrible customer service .,kp_0_5,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,...,kp_0_5,horrible customer service <SEP> negative,"[experience, employees]","[uninviting, rude]","[uninviting experience, rude employees]","[negative, negative]",customer service,horrible,['horrible customer service'],['negative']
4,302U8RURJZ1WH0D09XZ8R7HSOP5NVW,arg_0_537,0,It had wonderful acoustics .,kp_0_0,"[uninviting experience <SEP> negative, rude employees <SEP> negative]",Employees are rude and make the experience uninviting .,"[restaurant -> atmosphere, staff]",Arts & Entertainment,True,...,kp_0_0,wonderful acoustics <SEP> positive,"[experience, employees]","[uninviting, rude]","[uninviting experience, rude employees]","[negative, negative]",acoustics,wonderful,['wonderful acoustics'],['positive']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6794,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Great family owned restaurant .,kp_4_25,[friendly staff <SEP> positive],The staff are friendly and attentive .,[staff],Restaurants,False,...,kp_4_25,great family owned restaurant <SEP> positive,[staff],[friendly],[friendly staff],[positive],restaurant,great family owned,['great family owned restaurant'],['positive']
6795,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Worst service in the world !,kp_4_27,[friendly staff <SEP> positive],The staff are friendly and attentive .,[staff],Restaurants,False,...,kp_4_27,worst service <SEP> negative,[staff],[friendly],[friendly staff],[positive],service,worst,['worst service'],['negative']
6796,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Rude and inattentive staff .,kp_4_30,[friendly staff <SEP> positive],The staff are friendly and attentive .,[staff],Restaurants,False,...,kp_4_30,inattentive staff <SEP> negative,[staff],[friendly],[friendly staff],[positive],staff,inattentive,['inattentive staff'],['negative']
6797,3ZXV7Q5FJBOOUZUJKSLQTAKU1UIFC3,arg_4_6440,0,Great sushi and sashimi .,kp_4_31,[friendly staff <SEP> positive],The staff are friendly and attentive .,[st

In [8]:
df['label'].value_counts()

0    6380
1     419
Name: label, dtype: int64

In [9]:
def prepare_comment_kp_label_input(df):
    df['num_of_aspects'] = df['comment'].str.len()
    df = df.rename(columns={'comment_id': 'comment_id_sent'})
    df['comment_id'] = df['comment_id_sent'] + "_" + df['num_of_aspects'].astype(str)
    
    comment_df = df[['comment_id', 'topic', 'full_comment', 'num_of_aspects',
                'aspects_x', 'opinions_x', 'opinion_aspect_pairs_x', 'sentiments_x', 'attributes']]
    comment_df = comment_df.drop_duplicates(subset=['comment_id']).reset_index(drop=True)
    comment_df.columns = [col.replace("_x", "") for col in comment_df.columns]
    
    kp_df = df[['key_point_id', 'topic', 'full_key_point',
           'aspects_y', 'opinions_y', 'opinion_aspect_pairs_y', 'sentiments_y', 'attributes']]
    kp_df = kp_df.drop_duplicates(subset=['key_point_id']).reset_index(drop=True)
    kp_df = kp_df.rename(columns={'full_key_point': 'key_point'})
    kp_df.columns = [col.replace("_y", "") for col in kp_df.columns]
    kp_df['opinion_aspect_pairs'] = kp_df['opinion_aspect_pairs'].apply(lambda x: ast.literal_eval(x))
    kp_df['sentiments'] = kp_df['sentiments'].apply(lambda x: ast.literal_eval(x))
    
    labels_df = df[['comment_id', 'key_point_id', 'label']]
    
    return comment_df, kp_df, labels_df

# Comment-KP ABSA Matching

In [10]:
import spacy
nlp = spacy.load('en_core_web_md')

2023-07-12 11:38:21.470314: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-07-12 11:38:22.424055: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-07-12 11:38:22.424654: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-12 11:38:22.424883: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.77GHz coreCount: 80 deviceMemorySize: 12.00GiB deviceMemoryBandwidth: 849.46GiB/s
2023-07-12 11:38:22.424965: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-07-12 11:38:22.425001: I tensorflow/stream_ex

In [11]:
def comment_kp_absa_match(comment_df, kp_df):    
    comment_df = comment_df.explode(['aspects', 'opinions', 'opinion_aspect_pairs', 'sentiments', 'attributes'])
    kp_df = kp_df.explode(['opinion_aspect_pairs', 'sentiments'])
    
    # Matching based on ABSA predictions of comment and KPs
    sent_kp_df = comment_df.astype(str).merge(kp_df.astype(str), how='left', on=['topic']).dropna(subset=['key_point'])
    sent_kp_df = sent_kp_df.reset_index(drop=True)
    # Label the matching/non-matching pairs
    sent_kp_df.loc[
        (sent_kp_df['aspects_x'] == sent_kp_df['aspects_y']) & (sent_kp_df['sentiments_x'] == sent_kp_df['sentiments_y']),
        'score'
    ] = 1
    sent_kp_df.loc[
        (sent_kp_df['sentiments_x'] != sent_kp_df['sentiments_y']),
        'score'
    ] = 0
    sent_kp_df = implicit_aspects_matching(sent_kp_df)
    
    predictions_df = sent_kp_df.groupby(['topic', 'comment_id', 'opinion_aspect_pairs_x']).apply(select_best_kp_per_opinion).reset_index(drop=True)
    predictions_df = predictions_df.drop_duplicates(subset=['comment_id', 'key_point_id'])
    
    return predictions_df

In [12]:
def select_best_kp_per_opinion(grp):
    best_match = grp.sort_values(by=['score'], ascending=False)
    best_match = best_match.iloc[0]
    return best_match

Comment/KP matching on implicit aspects

In [13]:
# import spacy
# spacy.require_gpu()

In [14]:
def implicit_aspects_matching(sent_kp_df):
    sent_kp_df['predicted_by_cosine'] = False
    
    import warnings
    warnings.filterwarnings("error")
    implicit_match_analyze_mask = pd.isnull(sent_kp_df['score'])
    sent_kp_df[implicit_match_analyze_mask] = sent_kp_df[implicit_match_analyze_mask].apply(calculate_aspects_semantic_similarity, axis=1)
    warnings.filterwarnings("ignore")
    
    return sent_kp_df

In [15]:
unfound_tokens = []
def calculate_aspects_semantic_similarity(row):
    tokens = nlp(row['aspects_x'] + " | " + row['aspects_y'])
    sep_index = [token.i for token in tokens if token.text == '|'][0]
    token1, token2 = tokens[:sep_index], tokens[sep_index+1:]
    row['aspects_x_len'] = len(token1)
    row['aspects_y_len'] = len(token2)
#     print(token1, token2)
    try:
        row['score'] = token2.similarity(token1)
    except:
#         display(row)
#         unfound_tokens += [token1]
        row['score'] = 0
    row['predicted_by_cosine'] = True
    
    return row

In [16]:
import nltk
def label_implicit_matching_pair(row):
    token1 = row['aspects_x'][0]
    token2 = row['aspects_y'][0]
    
    # Best
    if row['aspects_x_len'] >= row['aspects_y_len']:
        if token2 in token1 and len(token1) > len(token2) and row['aspects_x_len'] > row['aspects_y_len']: # e.g. mexican food vs food
            row['label'] = 1
    return row

In [17]:
def do_eval(df):
    comment_df, kp_df, labels_df = prepare_comment_kp_label_input(df)
    predictions_df = comment_kp_absa_match(comment_df, kp_df)

    merged_df = pd.merge(predictions_df, labels_df, how="left", on=["comment_id", "key_point_id"])
    merged_df.loc[merged_df['key_point_id'] == "dummy_id", 'label'] = 0
    merged_df["label_strict"] = merged_df["label"].fillna(0)
    merged_df["label_relaxed"] = merged_df["label"].fillna(1)
    
    precisions = [(t, get_ap(group, "label_strict")) for t, group in merged_df.groupby(["topic"])]
    
    return merged_df, precisions

# Evaluation

## All Comments in test data

In [18]:
merged_df, precisions = do_eval(df)

In [19]:
precisions

[('Arts & Entertainment', 0.92),
 ('Automotive', 0.5822776003554035),
 ('Beauty & Spas', 0.8496957587866678),
 ('Hotels', 0.9538051289328429),
 ('Restaurants', 0.7765644827355906)]

## Comments containing multiple opinions in test data

In [20]:
merged_df, precisions = do_eval(df[df['isMultiAspect'] == True])

In [21]:
precisions

[('Arts & Entertainment', 0.8888888888888888),
 ('Automotive', 0.42857142857142855),
 ('Beauty & Spas', 0.823639455782313),
 ('Hotels', 0.879320987654321),
 ('Restaurants', 0.716017316017316)]

# Quantitative KP Coverage and Precision/Recall

In [22]:
merged_df, precisions = do_eval(df)

In [23]:
from sklearn.metrics import recall_score, precision_score

def get_top_kp_coverages(merged_df, df, category, top=None, selected_kps=[]):
    comment_df, kp_df, labels_df = prepare_comment_kp_label_input(df)
    kp_df = kp_df.drop(columns=['key_point'])

    valid_df = df[(df['label'] == 1) & (df['topic'] == category)]
    valid_kp = valid_df['full_key_point'].value_counts()
    
    top_kp = merged_df[merged_df['topic'] == category].merge(kp_df, on=['key_point_id'])['key_point'].value_counts()
    result = top_kp[top_kp.index.isin(valid_kp.index.tolist())]
    if top != None:
        result = result[result.index.isin(valid_kp.head(top).index)]
    elif len(selected_kps) > 0:
        result = result[result.index.isin(selected_kps)]
    
    if top != None:
        valid_df = df[df['full_key_point'].isin(valid_kp.head(top).index)]
    elif len(selected_kps) > 0:
        valid_df = valid_df[valid_df['full_key_point'].isin(selected_kps)]
    
    pred_df = merged_df[merged_df['topic'] == category].merge(kp_df, on=['key_point_id'])
    if top != None:
        pred_df = pred_df[pred_df['key_point'].isin(valid_kp.head(top).index)]
    elif len(selected_kps) > 0:
        pred_df = pred_df[pred_df['key_point'].isin(selected_kps)]
        
    pred_df['comment_id'] = pred_df['comment_id'].apply(lambda x: "_".join(x.split("_")[0:3]))
        
    return valid_df, pred_df

In [24]:
def calculate_precision_recall(valid_df, pred_df):
    pred_df['pred_label'] = 1
    
    # Calculate Precision
    new_valid_df = valid_df.merge(pred_df[['comment_id', 'key_point_id', 'pred_label']], how='right')
    new_valid_df['label'] = new_valid_df['label'].fillna(0)
    precision = precision_score(new_valid_df['label'], new_valid_df['pred_label'])

    # Calculate Recall
    new_valid_df = valid_df.merge(pred_df[['comment_id', 'key_point_id', 'pred_label']], how='left')
    new_valid_df['pred_label'] = new_valid_df['pred_label'].fillna(0)
    recall = recall_score(new_valid_df['label'], new_valid_df['pred_label'])
    
    return precision, recall

In [25]:
def generate_coverage_table(valid_df, pred_df):
    coverage_compare_table = valid_df['full_key_point'].value_counts().reset_index()
    coverage_compare_table = coverage_compare_table.rename(columns={'full_key_point': 'Actual Coverage'})
    coverage_compare_table = coverage_compare_table.merge(pred_df['key_point'].value_counts().reset_index(), on='index', how='left')
    coverage_compare_table = coverage_compare_table.rename(columns={'key_point': 'Predicted Coverage', 'index': 'Key Point'})
    coverage_compare_table['Predicted Coverage'] = coverage_compare_table['Predicted Coverage'].fillna(0)
    
    return coverage_compare_table[['Key Point', 'Predicted Coverage', 'Actual Coverage']]

## Arts & Entertainment

In [26]:
selected_kps = ['Friendly and helpful staff .', 'Seats are adequately comfortable .   ', 'Horrible customer service .']
valid_df, pred_df = get_top_kp_coverages(merged_df, df, 'Arts & Entertainment', None, selected_kps)

In [27]:
generate_coverage_table(valid_df, pred_df)

,Key Point,Predicted Coverage,Actual Coverage
0,Friendly and helpful staff .,10,14
1,Seats are adequately comfortable .,4,4
2,Horrible customer service .,3,3


In [28]:
calculate_precision_recall(valid_df, pred_df)

(0.9411764705882353, 0.7619047619047619)

## Automotive

In [29]:
selected_kps = ['They have excellent customer service .', 'The employees here are wonderful !', 'Very professional staff .']
valid_df, pred_df = get_top_kp_coverages(merged_df, df, 'Automotive', None, selected_kps)

In [30]:
generate_coverage_table(valid_df, pred_df)

,Key Point,Predicted Coverage,Actual Coverage
0,They have excellent customer service .,10.0,29
1,Very professional staff .,0.0,13
2,The employees here are wonderful !,2.0,13


In [31]:
calculate_precision_recall(valid_df, pred_df)

(0.8333333333333334, 0.18181818181818182)

## Beauty & Spas

In [32]:
selected_kps = ['Staff is friendly and accommodating .', 'Customer service- EXCELLENT !', 'Amazing & professional service .']
valid_df, pred_df = get_top_kp_coverages(merged_df, df, "Beauty & Spas", None, selected_kps)

In [33]:
generate_coverage_table(valid_df, pred_df)

,Key Point,Predicted Coverage,Actual Coverage
0,Staff is friendly and accommodating .,13,18
1,Amazing & professional service .,3,14
2,Customer service- EXCELLENT !,7,13


In [34]:
calculate_precision_recall(valid_df, pred_df)

(0.8695652173913043, 0.4444444444444444)

## Hotels

In [35]:
selected_kps = ['Friendly and helpful staff .', 'Clean and comfortable rooms .', 'The ambience is wonderfully peaceful .']
valid_df, pred_df = get_top_kp_coverages(merged_df, df, 'Hotels', None, selected_kps)

In [36]:
generate_coverage_table(valid_df, pred_df)

,Key Point,Predicted Coverage,Actual Coverage
0,Friendly and helpful staff .,16,21
1,Clean and comfortable rooms .,12,13
2,The ambience is wonderfully peaceful .,2,1


In [37]:
calculate_precision_recall(valid_df, pred_df)

(0.9333333333333333, 0.8)

## Restaurants

In [38]:
selected_kps = ['Staff was courteous and accommodating .', 
                'Fresh food , using local produce .', 'The service here was exceptional .  ']
valid_df, pred_df = get_top_kp_coverages(merged_df, df, 'Restaurants', None, selected_kps)

In [39]:
generate_coverage_table(valid_df, pred_df)

,Key Point,Predicted Coverage,Actual Coverage
0,Staff was courteous and accommodating .,11,19
1,"Fresh food , using local produce .",8,5
2,The service here was exceptional .,5,5


In [40]:
calculate_precision_recall(valid_df, pred_df)

(0.75, 0.6206896551724138)